In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp


In [2]:
# === Load and prepare data ===
train_df = pd.read_csv('./data/train.csv', header=None)
test_df = pd.read_csv('./data/test.csv', header=None)

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Columns: 201 entries, 0 to 200
dtypes: float64(201)
memory usage: 13.0 MB


In [3]:
# Split label and features

y_all = train_df.iloc[:, 0].values
X_all = train_df.iloc[:, 1:].values
y_test = test_df.iloc[:, 0].values
X_test = test_df.iloc[:, 1:].values

# Convert labels from {0,1} to {-1,1}
y_all = 2 * y_all - 1
y_test = 2 * y_test - 1

# Split into training (first 4000) and validation (rest)
X_train = X_all[:4000]
y_train = y_all[:4000]
X_val = X_all[4000:]
y_val = y_all[4000:]


### Question 2: Primal Soft Margin

$\min_{w,b,\zeta_i} \quad \frac{1}{2}||w||^2_2 + C\sum_i^m\zeta_i$

$s.t. \quad y_i(x^Tx+b) \geq 1 - \zeta_i \quad \zeta_i\geq 0$

In [4]:
def svm_train_primal(data_train, label_train, c_value=1):
    N = data_train.shape[0] # number of samples
    d = data_train.shape[1] # number of features
    
    C = cp.Parameter(nonneg=True)
    C.value = c_value
    w = cp.Variable(d)
    b = cp.Variable()
    zeta = cp.Variable(N)

    # Objective: (1/2)||w||^2 + C/N * \sum_i^N \zeta_i
    objective = cp.Minimize(0.5 * cp.sum_squares(w) + C/N * cp.sum(zeta))

    # Constraints: y_i(w^T x_i + b) ≥ 1 - \zeta_i
    constraints = [
        cp.multiply(label_train, data_train @ w + b) >= 1 - zeta,
        zeta >= 0
    ]

    prob = cp.Problem(objective, constraints)
    prob.solve()

    return {'w': w.value, 'b': b.value}

def svm_predict_primal(data, labels, model):
    w = model['w']
    b = model['b']
    y_pred = np.sign(data @ w + b)
    accuracy = np.mean(y_pred == labels)
    return accuracy



In [5]:
# Train SVM on first 4000 samples
C = 100
model = svm_train_primal(X_train, y_train, C)

# Evaluate
val_acc = svm_predict_primal(X_val, y_val, model)
test_acc = svm_predict_primal(X_test, y_test, model)

# Report
print("Bias term b:", model['b'])
print("Sum of weights np.sum(w):", np.sum(model['w']))
print("Validation Accuracy:", val_acc)
print("Test Accuracy:", test_acc)

Bias term b: 1.779737104796226
Sum of weights np.sum(w): -0.1461666248360982
Validation Accuracy: 0.9695555555555555
Test Accuracy: 0.968


### Question 3: Dual Soft Margin

$\max_{\alpha_i}\quad \sum_i\alpha_i - \frac{1}{2}\sum_i\sum_j\alpha_i\alpha_jy_iy_jx_ix_j$

$s.t. \quad \sum_i\alpha_iy_i=0; 0\leq\alpha_i\leq C$

In [6]:
def svm_train_dual(data_train, label_train, c_value=1):
    X = data_train
    y = label_train
    N = X.shape[0]
    d = X.shape[1]
    
    Y = np.diag(y)
    C = cp.Parameter(nonneg=True)
    C.value = c_value
    alpha = cp.Variable(N)
    P = cp.atoms.affine.wraps.psd_wrap(Y @ X @ X.T @ Y) # Using asymmetric matrix for quadratic form
    
    # Define the exact dual objective 
    # L = \sum_i^N \alpha_i - 0.5 * \sum_{i,j}^N \alpha_i \alpha_j y_i y_j (x_i^T x_j)
    # Must convert to Associated symmetric matrix form for the quadratic form part of the dual problem
    # Any other, such as original inner sum_squares, will not work with cvxpy that raising complexity issues
    # and computational cost.
    # L = \sum_i^N \alpha_i - 0.5 * \alpha^T P \alpha
    obj = cp.sum(alpha) - 0.5 * cp.quad_form(alpha, P)

    # Constraints: 0 =< \alpha_i <= C/N, sum(\alpha_i, y_i) = 0
    constraints = [
        alpha >= 0,
        alpha <= C / N,
        cp.sum(cp.multiply(alpha, y)) == 0
    ]

    # Solve the problem
    prob = cp.Problem(cp.Maximize(obj), constraints)
    prob.solve()

    alpha_val = alpha.value

    return {'alpha': alpha_val}



In [7]:
# Train SVM on first 4000 samples
C = 100
model = svm_train_dual(X_train, y_train, C)

print("Sum of Lagrange multipliers (np.sum(alpha)):", np.sum(model['alpha']))

Sum of Lagrange multipliers (np.sum(alpha)): 7.283433929638309


### Question 4

In [8]:
# Recover sum of weights and the bias from dual multipliers \alpha_i

diag_y = np.diag(y_train)
alpha = model['alpha']

w = X_train.T @ diag_y @ alpha
b = cp.mean(y_train - X_train @ w)

print(f"Sum of weights is: {np.sum(w)}\nBias term b: {b.value}")


Sum of weights is: -0.14481770546687656
Bias term b: 1.7933270614830095


### Question 5

All $\alpha_i > 0$ points are support vectors. They can be classified as:

1. On the margin: $y_i(w^T x_i + b) = 1$ $\Leftarrow$ $\zeta_i = 0$
2. Within the margin: $0 < y_i(w^T x_i + b) < 1$ $\Leftarrow$ $0 < \zeta_i < 1$
3. Misclassified: $y_i(w^T x_i + b) < 0$ $\Leftarrow$ $\zeta_i > 1$

In [9]:
def get_support_vectors_from_primal(model, X, y, tol=1e-5):
    """
    Identify support vectors from the primal SVM solution.
    
    Parameters:
    model : dict
        Dictionary containing 'w' and 'b' from svm_train_primal
    X : ndarray
        Training data matrix (n_samples x n_features)
    y : ndarray
        Training labels (+1 or -1)
    tol : float, optional
        Numerical tolerance for margin comparison
        
    Returns:
    support_indices : ndarray
        Indices of support vectors in the training data
    support_vectors : ndarray
        Support vector data points
    support_labels : ndarray
        Labels of support vectors
    """
    w = model['w']
    b = model['b']
    
    # Calculate margin distances: y_i(w·x_i + b)
    margins = y * (X @ w + b)
    
    # Identify support vectors: points where margin <= 1 + tolerance
    support_indices = np.where(margins <= 1 + tol)[0]
    support_vectors = X[support_indices]
    support_labels = y[support_indices]
    
    return support_indices, support_vectors, support_labels

# Assuming you have training data (X_train, y_train)
primal_model = svm_train_primal(X_train, y_train, c_value=1.0)

# Get support vectors
sv_idx, sv, sv_labels = get_support_vectors_from_primal(
    primal_model, X_train, y_train
)

print(f"Found {len(sv_idx)} support vectors:")
print(f"Indices: {sv_idx}")
# print(f"Support vectors:\n{sv}")
# print(f"Support labels: {sv_labels}")

Found 1367 support vectors:
Indices: [   0    6   11 ... 3991 3993 3998]


In [10]:
def save_support_vectors_to_csv(support_indices, support_vectors, support_labels, filename):
    df = pd.DataFrame(support_vectors)
    df.insert(0, 'label', support_labels)
    df.insert(0, 'index', support_indices)
    df.to_csv(filename, index=False)
    print(f"Support vectors saved to: {filename}")

# Save to file
save_support_vectors_to_csv(sv_idx, sv, sv_labels, "./data/support_vectors.csv")

Support vectors saved to: ./data/support_vectors.csv


### Question 6

From KKT conditions:

1. Margin support vectors: $0 < \alpha_i < C$ (lie exactly on margin)
2. Bound support vectors: $\alpha_i = C$ (may be misclassified/margin violators)

In [11]:
def get_support_vectors_from_dual(model, X: np.ndarray, y: np.ndarray, C, tol=1e-5):
    """
    Identify support vectors from the dual SVM solution.
    
    Parameters:
    model : dict
        Dictionary containing 'alpha' from svm_train_dual
    X : ndarray
        Training data matrix (n_samples x n_features)
    y : ndarray
        Training labels (+1 or -1)
    tol : float, optional
        Numerical tolerance for alpha comparison
        
    Returns:
    support_indices : ndarray
        Indices of support vectors in the training data
    support_vectors : ndarray
        Support vector data points
    support_labels : ndarray
        Labels of support vectors
    margin_vectors : ndarray
        Indices of vectors exactly on the margin (0 < alpha < C/N)
    bound_vectors : ndarray
        Indices of vectors at the bound (alpha = C/N)
    """
    alpha = model['alpha']
    C = C
    N = X.shape[0]
    
    # Identify support vectors (alpha > 0)
    support_mask = alpha > tol
    support_indices = np.where(support_mask)[0]
    support_vectors = X[support_mask]
    support_labels = y[support_mask]
    
    # Classify support vectors into margin vectors and bound vectors
    margin_mask = (alpha > tol) & (alpha < C/N - tol)
    bound_mask = (alpha > tol) & (alpha >= C/N - tol)
    
    margin_vectors = np.where(margin_mask)[0]
    bound_vectors = np.where(bound_mask)[0]
    
    return {
        'support_indices': support_indices,
        'support_vectors': support_vectors,
        'support_labels': support_labels,
        'margin_vectors': margin_vectors,  # Exactly on the margin
        'bound_vectors': bound_vectors    # At the upper bound (potential margin violators)
    }


In [12]:
# Get support vectors
sv_info = get_support_vectors_from_dual(model, X_train, y_train, C=C)

print(f"Total support vectors: {len(sv_info['support_indices'])})")
print(f"Margin vectors (0 < alpha < C/N): {len(sv_info['margin_vectors'])})")
print(f"Bound vectors (alpha = C/N): {len(sv_info['bound_vectors'])})")

Total support vectors: 392)
Margin vectors (0 < alpha < C/N): 179)
Bound vectors (alpha = C/N): 213)


In [13]:
save_support_vectors_to_csv(sv_info['support_indices'], sv_info['support_vectors'], sv_info['support_labels'], "./data/support_vectors_dual.csv")

Support vectors saved to: ./data/support_vectors_dual.csv


### Question 7

In [14]:
# define regularization C range to find the optimal value
regularization_para_C = [2 ** i for i in range(-10, 11, 2)]
print("C candidates:", regularization_para_C)

best_val_acc = -1
best_C = None
best_model = None

# Start training the SVM and find the best C on validation set
for C in regularization_para_C:
    svm_model = svm_train_primal(X_train, y_train, C)
    val_accuracy = svm_predict_primal(X_val, y_val, svm_model)
    test_accuracy = svm_predict_primal(X_test, y_test, svm_model)
    # print(f"Val accuracy with C={C}: {val_accuracy}")
    # print(f"Test accuracy with C={C}: {test_accuracy}")
    # print()
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        best_C = C
        best_model = svm_model

print(f"Best C found on validation set: {best_C}")
print(f"Validation accuracy with best C:{best_C}: {best_val_acc}")
# Report test accuracy using the optimal C
best_test_acc = svm_predict_primal(X_test, y_test, best_model)
print(f"Test accuracy with best C: {best_test_acc}")


C candidates: [0.0009765625, 0.00390625, 0.015625, 0.0625, 0.25, 1, 4, 16, 64, 256, 1024]
Best C found on validation set: 4
Validation accuracy with best C:4: 0.9748888888888889
Test accuracy with best C: 0.9746666666666667


### Question 8

In [15]:
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from libsvm.svmutil import svm_train, svm_predict, svm_problem, svm_parameter


In [16]:
# Define C range as required
C_range = [2 ** i for i in range(-10, 11, 2)]

# --- scikit-learn SVM ---
best_val_acc_sklearn = -1
best_C_sklearn = None
best_model_sklearn = None

for C in C_range:
    clf = LinearSVC(C=C, max_iter=10000, dual=True) # may unnecessary care the non-convergence warning; the result may not be influenced. Even increase the max_ite to 100k, the convergence still failed but result always same
    clf.fit(X_train, y_train)
    val_acc = clf.score(X_val, y_val)
    if val_acc > best_val_acc_sklearn:
        best_val_acc_sklearn = val_acc
        best_C_sklearn = C
        best_model_sklearn = clf

test_acc_sklearn = best_model_sklearn.score(X_test, y_test)
print(f"[scikit-learn] Best C: {best_C_sklearn}")
print(f"[scikit-learn] Validation accuracy: {best_val_acc_sklearn}")
print(f"[scikit-learn] Test accuracy: {test_acc_sklearn}")


/Applications/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Applications/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Applications/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Applications/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[scikit-learn] Best C: 0.00390625
[scikit-learn] Validation accuracy: 0.9666666666666667
[scikit-learn] Test accuracy: 0.968


/Applications/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [17]:
# --- LIBSVM ---
if 'svm_train' in globals():
    best_val_acc_libsvm = -1
    best_C_libsvm = None
    best_model_libsvm = None

    for C in C_range:
        param = svm_parameter(f'-t 0 -c {C} -q')
        prob = svm_problem(list(y_train), X_train.tolist())
        model = svm_train(prob, param)
        _, val_acc, _ = svm_predict(list(y_val), X_val.tolist(), model, options='-q')
        val_acc = val_acc[0] / 100.0  # LIBSVM returns percentage
        if val_acc > best_val_acc_libsvm:
            best_val_acc_libsvm = val_acc
            best_C_libsvm = C
            best_model_libsvm = model

    # Test accuracy with best C
    _, test_acc, _ = svm_predict(list(y_test), X_test.tolist(), best_model_libsvm, options='-q')
    test_acc = test_acc[0] / 100.0
    print(f"[LIBSVM] Best C: {best_C_libsvm}")
    print(f"[LIBSVM] Validation accuracy: {best_val_acc_libsvm}")
    print(f"[LIBSVM] Test accuracy: {test_acc}")
else:
    print("LIBSVM not available, skipping LIBSVM SVM classification.")
# run 2min55sec

[LIBSVM] Best C: 0.0009765625
[LIBSVM] Validation accuracy: 0.9746666666666667
[LIBSVM] Test accuracy: 0.9739999999999999
